# <center>Deep Learning and Text Analytics II</center>

References:
- General introduction
   - http://ufldl.stanford.edu/tutorial/supervised/MultiLayerNeuralNetworks/
- Word vector:
     - https://code.google.com/archive/p/word2vec/
- Keras tutorial
     - https://machinelearningmastery.com/tutorial-first-neural-network-python-keras/
- CNN
     - http://www.wildml.com/2015/11/understanding-convolutional-neural-networks-for-nlp/

## 1. Agenda
- Introduction to neural networks
- Word/Document Vectors (vector representation of words/phrases/paragraphs)
- Convolutional neural network (CNN)
- Application of CNN in text classification

## 4. Word2Vector (a.k.a word embedding) and Doc2Vector

### 4.1. Word2Vector
 - Vector representation of words (i.e. word vectors) learned using neural network
   - e.g. "apple" : [0.35, -0.2, 0.4, ...], 'mongo':  [0.32, -0.18, 0.5, ...]
   - Interesting properties of word vectors:
    * **Words with similar semantics have close word vectors**
    * **Composition**: e.g. vector("woman")+vector("king")-vector('man') $\approx$ vector("queen")
 - Models:
   - **CBOW** (Continuous Bag of Words): Predict a target word based on context
     - e.g. the fox jumped over the lazy dog
     - Assuming symmetric context with window size 3, this sentence can create training samples: 
       - ([-, fox], the) 
       - ([the, jumped], fox) 
       - ([fox, over], jumped)
       - ([jumped, the], over) 
       - ...
       
       <img src="cbow.png" width="50%">
       source: https://www.analyticsvidhya.com/blog/2017/06/word-embeddings-count-word2veec/
   - **Skip Gram**: predict context based on target words
   
        <img src="skip_gram.png" width="50%">
        source: https://www.analyticsvidhya.com/blog/2017/06/word-embeddings-count-word2veec/
   - Nagtive Sampling: 
       - When training a neural network, for each sample,  all weights are adjusted slightly so that it predicts that training sample more accurately. 
       - CBOW or skip gram models have tremendous number of weights, all of which would be updated slightly by every one of billions of training samples!
       - Negative sampling addresses this by having **each training sample only modify a small percentage of the weights, rather than all of them**. 
       - e.g. when training with sample ([fox, over], jumped), update output weights connected to "jumped" along with a small number of other "negative words" sampled randomly
       - For details, check http://mccormickml.com/2017/01/11/word2vec-tutorial-part-2-negative-sampling/
   
 

In [1]:
# set up interactive shell
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [5]:
# Exercise 4.1.1 Train your word vector

import pandas as pd
import nltk,string

# Load data
data=pd.read_csv('../amazon_review_large.csv', header=None)
data.columns=['label','text']
data.head()

# tokenize each document into a list of unigrams
# strip punctuations and leading/trailing spaces from unigrams
# only unigrams with 2 or more characters are taken
sentences=[ [token.strip(string.punctuation).strip() \
             for token in nltk.word_tokenize(doc.lower()) \
                 if token not in string.punctuation and \
                 len(token.strip(string.punctuation).strip())>=2]\
             for doc in data["text"]]
print(sentences[0:2])

,label,text
0,2,This is a little longer and more detailed than...
1,1,Only Michelle Branch save this album!!!!All gu...
2,2,"A surprisingly good book, given its inherently..."
3,2,"This is a wonderful, quiet and relaxing CD tha..."
4,1,The lights that I received are absolutely not ...


[['this', 'is', 'little', 'longer', 'and', 'more', 'detailed', 'than', 'the', 'first', 'two', 'books', 'in', 'the', 'series', 'however', 'have', 'enjoyed', 'each', 'new', 'aspect', 'of', 'the', 'exciting', 'fantasy', 'universe'], ['only', 'michelle', 'branch', 'save', 'this', 'album', 'all', 'guys', 'play', 'along', 'with', 'unenthusiastic', 'beat', 'even', 'karl']]


In [6]:
# Train your own word vectors using gensim

# gensim.models is the package for word2vec
# check https://radimrehurek.com/gensim/models/word2vec.html
# for detailed description

from gensim.models import word2vec
import logging
import pandas as pd

# print out tracking information
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', \
                    level=logging.INFO)

# min_count: words with total frequency lower than this are ignored
# size: the dimension of word vector
# window: context window, i.e. the maximum distance 
#         between the current and predicted word 
#         within a sentence (i.e. the length of ngrams)
# workers: # of parallel threads in training
# for other parameters, check https://radimrehurek.com/gensim/models/word2vec.html
wv_model = word2vec.Word2Vec(sentences, \
            min_count=5, size=200, \
            window=5, workers=4 )

K:\Anaconda3\lib\site-packages\gensim\utils.py:1212: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")
2018-11-29 16:07:06,077 : INFO : collecting all words and their counts
2018-11-29 16:07:06,078 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2018-11-29 16:07:06,297 : INFO : PROGRESS: at sentence #10000, processed 712003 words, keeping 36988 word types
2018-11-29 16:07:06,580 : INFO : collected 55278 word types from a corpus of 1424321 raw words and 20000 sentences
2018-11-29 16:07:06,580 : INFO : Loading a fresh vocabulary
2018-11-29 16:07:06,647 : INFO : effective_min_count=5 retains 12133 unique words (21% of original 55278, drops 43145)
2018-11-29 16:07:06,647 : INFO : effective_min_count=5 leaves 1361983 word corpus (95% of original 1424321, drops 62338)
2018-11-29 16:07:06,714 : INFO : deleting the raw counts dictionary of 55278 items
2018-11-29 16:07:06,714 :

In [7]:
# test word2vec model

print("Top 5 words similar to word 'sound'")
wv_model.wv.most_similar('sound', topn=5)

print("Top 5 words similar to word 'sound' but not relevant to 'film'")
wv_model.wv.most_similar(positive=['sound','music'], \
                         negative=['film'], topn=5)

print("Similarity between 'movie' and 'film':")
wv_model.wv.similarity('movie','film') 

print("Similarity between 'movie' and 'city':")
wv_model.wv.similarity('movie','city') 

print("Word does not match with others in the list of \
['sound', 'music', 'graphics', 'actor', 'book']:")
wv_model.wv.doesnt_match(["sound", "music", \
                          "graphics", "actor", "book"])

print("Word vector for 'movie':")
wv_model.wv['movie']

2018-11-29 16:10:06,045 : INFO : precomputing L2-norms of word weight vectors


Top 5 words similar to word 'sound'


K:\Anaconda3\lib\site-packages\gensim\matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int32 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('metal', 0.768135666847229),
 ('sounds', 0.7587062120437622),
 ('vocals', 0.750717282295227),
 ('production', 0.74760901927948),
 ('rock', 0.7452607154846191)]

Top 5 words similar to word 'sound' but not relevant to 'film'


...

0.009261133

Word does not match with others in the list of ['sound', 'music', 'graphics', 'actor', 'book']:


'book'

Word vector for 'movie':


array([-0.77269727, -1.4913762 , -1.0019791 , -1.2563334 , -0.12225643,
       -1.416501  ,  1.2577945 , -0.5184871 ,  0.6585174 , -0.14731687,
        0.52918917,  0.30294475,  0.495267  ,  0.21776067,  0.124472  ,
       -1.5360495 ,  0.54938453, -0.06506   , -1.639378  , -0.24853028,
        0.7968809 , -1.2775854 ,  0.13571614, -0.5208313 ,  1.0953196 ,
       -0.3756403 , -1.5401527 , -1.8848073 , -0.9565676 , -0.81631154,
       -1.5816301 ,  0.8726593 , -1.9249322 , -0.27146304, -0.7126626 ,
        0.08017053, -1.3893516 ,  0.57443464,  1.6364825 , -1.101771  ,
       -0.86638355,  1.5470277 ,  2.1122425 , -0.57389414, -0.06188415,
       -0.53467387,  0.7612246 ,  0.25090617, -1.0429192 ,  0.80020404,
       -0.3509972 ,  1.3647641 ,  1.1475662 ,  0.23796152, -0.5293686 ,
       -0.89735174, -1.1483487 , -0.8870737 , -0.18562146,  1.0308938 ,
       -0.60114026, -0.04594025, -0.421823  ,  1.1857368 , -0.17878392,
        1.1128901 , -0.68922096,  0.6759488 , -0.18189444,  0.67

### 4.2. Pretrained Word Vectors
- Google published pre-trained 300-dimensional vectors for 3 million words and phrases that were trained on Google News dataset (about 100 billion words)(https://code.google.com/archive/p/word2vec/)
- GloVe (Global Vectors for Word Representation): Pretained word vectors from different data sources provided by Standford https://nlp.stanford.edu/projects/glove/
- FastText by Facebook https://github.com/facebookresearch/fastText/blob/master/pretrained-vectors.md

In [8]:
# Exercise 4.2.1: Use pretrained word vectors

# download the bin file for pretrained word vectors
# from above links, e.g. https://drive.google.com/file/d/0B7XkCwpI5KDYNlNUTTlSS21pQmM/edit?usp=sharing
# Warning: the bin file is very big (over 2G)
# You need a powerful machine to load it

import gensim

model = gensim.models.KeyedVectors.\
load_word2vec_format('GoogleNews-vectors-negative300.bin', binary=True) 

model.wv.most_similar(positive=['women','king'], \
                      negative='man')

2018-11-29 16:10:22,879 : INFO : loading projection weights from GoogleNews-vectors-negative300.bin


FileNotFoundError: [Errno 2] No such file or directory: 'GoogleNews-vectors-negative300.bin'

### 4.3. Sentence/Paragraph/Document Vectors
- So far we learned vector representation of words
- A lot of times, our samples are sentences, paragraphs, or documents
- How to create vector representations of sentences, paragraphs, or documents?
  - Weighted average of word vectors (however, word order is lost as "bag of words")
  - Concatenation of word vectors (large space)
  - ??
- Paragraph Vector: A distributed memory model (PV-DM)
   - Word vectors are shared across paragraphs
   - The paragraph vector is shared across all contexts generated from the same paragraph but not across paragraphs
   - **Both pragraph vectors and word vectors** are returned
   - Paragraph vectors can be used for document retrival or as features for classification or clustering
  <img src="doc2vec.png" width="50%">
   Source: Le Q. and Mikolov, T. Distributed Representations of Sentences and Documents https://arxiv.org/pdf/1405.4053v2.pdf

In [9]:
# Exercise 4.3.1 Train your word vector

# We have tokenized sentences
# Label each sentence with a unique tag

from gensim.models.doc2vec import TaggedDocument

docs=[TaggedDocument(sentences[i], [str(i)]) for i in range(len(sentences)) ]
docs[0]

TaggedDocument(words=['this', 'is', 'little', 'longer', 'and', 'more', 'detailed', 'than', 'the', 'first', 'two', 'books', 'in', 'the', 'series', 'however', 'have', 'enjoyed', 'each', 'new', 'aspect', 'of', 'the', 'exciting', 'fantasy', 'universe'], tags=['0'])

In [10]:
from random import shuffle

# package for doc2vec
from gensim.models import doc2vec

# for more parameters, check
# https://radimrehurek.com/gensim/models/doc2vec.html

# initialize the model without documents
# distributed memory model is used (dm=1)
model = doc2vec.Doc2Vec(dm=1, min_count=5, window=5, size=200, workers=4)

# build the vocabulary using the documents
model.build_vocab(docs) 

# train the model in 20 epoches
# You may need to incease epoches
for epoch in range(30):
    # shuffle the documents in each epoch
    shuffle(docs)
    # in each epoch, all samples are used
    model.train(docs, total_examples=len(docs), epochs=1)

K:\Anaconda3\lib\site-packages\gensim\models\doc2vec.py:570: UserWarning: The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.
  warnings.warn("The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.")
2018-11-29 16:46:03,112 : INFO : collecting all words and their counts
2018-11-29 16:46:03,113 : INFO : PROGRESS: at example #0, processed 0 words (0/s), 0 word types, 0 tags
2018-11-29 16:46:03,372 : INFO : PROGRESS: at example #10000, processed 712003 words (2667276/s), 36988 word types, 10000 tags
2018-11-29 16:46:03,756 : INFO : collected 55278 word types and 20000 unique tags from a corpus of 20000 examples and 1424321 words
2018-11-29 16:46:03,756 : INFO : Loading a fresh vocabulary
2018-11-29 16:46:03,821 : INFO : effective_min_count=5 retains 12133 unique words (21% of original 55278, drops 43145)
2018-11-29 16:46:03,821 : INFO : effective_min_count=5 leaves 1361983 word corpus (95% of original 1424321, drops 62

2018-11-29 16:46:28,615 : INFO : worker thread finished; awaiting finish of 3 more threads
2018-11-29 16:46:28,618 : INFO : worker thread finished; awaiting finish of 2 more threads
2018-11-29 16:46:28,629 : INFO : worker thread finished; awaiting finish of 1 more threads
2018-11-29 16:46:28,640 : INFO : worker thread finished; awaiting finish of 0 more threads
2018-11-29 16:46:28,642 : INFO : EPOCH - 1 : training on 1424321 raw words (1035937 effective words) took 3.9s, 264299 effective words/s
2018-11-29 16:46:28,643 : INFO : training on a 1424321 raw words (1035937 effective words) took 3.9s, 263277 effective words/s
2018-11-29 16:46:28,665 : WARNING : Effective 'alpha' higher than previous training cycles
2018-11-29 16:46:28,665 : INFO : training model with 4 workers on 12133 vocabulary and 200 features, using sg=0 hs=0 sample=0.001 negative=5 window=5
2018-11-29 16:46:29,683 : INFO : EPOCH 1 - PROGRESS: at 30.60% examples, 316154 words/s, in_qsize 8, out_qsize 0
2018-11-29 16:46:3

2018-11-29 16:46:56,194 : INFO : worker thread finished; awaiting finish of 0 more threads
2018-11-29 16:46:56,197 : INFO : EPOCH - 1 : training on 1424321 raw words (1035689 effective words) took 5.4s, 191307 effective words/s
2018-11-29 16:46:56,198 : INFO : training on a 1424321 raw words (1035689 effective words) took 5.4s, 190723 effective words/s
2018-11-29 16:46:56,249 : WARNING : Effective 'alpha' higher than previous training cycles
2018-11-29 16:46:56,249 : INFO : training model with 4 workers on 12133 vocabulary and 200 features, using sg=0 hs=0 sample=0.001 negative=5 window=5
2018-11-29 16:46:57,272 : INFO : EPOCH 1 - PROGRESS: at 16.76% examples, 171964 words/s, in_qsize 7, out_qsize 0
2018-11-29 16:46:58,436 : INFO : EPOCH 1 - PROGRESS: at 34.19% examples, 163612 words/s, in_qsize 8, out_qsize 0
2018-11-29 16:46:59,449 : INFO : EPOCH 1 - PROGRESS: at 52.92% examples, 172926 words/s, in_qsize 7, out_qsize 0
2018-11-29 16:47:00,459 : INFO : EPOCH 1 - PROGRESS: at 70.41% ex

2018-11-29 16:47:24,865 : INFO : training on a 1424321 raw words (1035325 effective words) took 4.3s, 238241 effective words/s
2018-11-29 16:47:24,896 : WARNING : Effective 'alpha' higher than previous training cycles
2018-11-29 16:47:24,896 : INFO : training model with 4 workers on 12133 vocabulary and 200 features, using sg=0 hs=0 sample=0.001 negative=5 window=5
2018-11-29 16:47:25,916 : INFO : EPOCH 1 - PROGRESS: at 16.21% examples, 164994 words/s, in_qsize 8, out_qsize 0
2018-11-29 16:47:26,967 : INFO : EPOCH 1 - PROGRESS: at 39.67% examples, 199648 words/s, in_qsize 8, out_qsize 0
2018-11-29 16:47:28,020 : INFO : EPOCH 1 - PROGRESS: at 67.48% examples, 225806 words/s, in_qsize 8, out_qsize 0
2018-11-29 16:47:29,020 : INFO : EPOCH 1 - PROGRESS: at 95.69% examples, 241206 words/s, in_qsize 7, out_qsize 0
2018-11-29 16:47:29,116 : INFO : worker thread finished; awaiting finish of 3 more threads
2018-11-29 16:47:29,138 : INFO : worker thread finished; awaiting finish of 2 more thread

2018-11-29 16:47:48,876 : INFO : EPOCH - 1 : training on 1424321 raw words (1035510 effective words) took 5.4s, 190150 effective words/s
2018-11-29 16:47:48,878 : INFO : training on a 1424321 raw words (1035510 effective words) took 5.5s, 189861 effective words/s
2018-11-29 16:47:48,932 : WARNING : Effective 'alpha' higher than previous training cycles
2018-11-29 16:47:48,932 : INFO : training model with 4 workers on 12133 vocabulary and 200 features, using sg=0 hs=0 sample=0.001 negative=5 window=5
2018-11-29 16:47:49,957 : INFO : EPOCH 1 - PROGRESS: at 14.85% examples, 151606 words/s, in_qsize 7, out_qsize 0
2018-11-29 16:47:50,969 : INFO : EPOCH 1 - PROGRESS: at 31.93% examples, 161650 words/s, in_qsize 8, out_qsize 0
2018-11-29 16:47:51,977 : INFO : EPOCH 1 - PROGRESS: at 48.53% examples, 165103 words/s, in_qsize 8, out_qsize 0
2018-11-29 16:47:52,986 : INFO : EPOCH 1 - PROGRESS: at 67.44% examples, 172461 words/s, in_qsize 8, out_qsize 0
2018-11-29 16:47:54,002 : INFO : EPOCH 1 - 

In [11]:
# Inspect paragraph vectors and word vectors

# the pragraph vector of the first document
model.docvecs['0']

# the word vector of 'movie'
model.wv['movie']

array([-0.16183381,  0.12456312, -0.01335587, -0.42504102, -0.30003437,
       -0.0140024 ,  0.03075002, -0.07262519,  0.16955574, -0.00588711,
        0.13330661,  0.34285158,  0.03422689,  0.16842516, -0.25482705,
       -0.19358243,  0.04123701, -0.13208947, -0.35571748,  0.12064377,
       -0.30898485, -0.29411885, -0.16552879, -0.15874691,  0.04507516,
       -0.04300391,  0.32410267,  0.013937  , -0.04040687, -0.29224434,
       -0.04856979, -0.1116295 ,  0.00915188,  0.09112127, -0.2545681 ,
       -0.3345861 , -0.07077353,  0.31784013,  0.04960489, -0.41506338,
       -0.01637752, -0.22232306, -0.09818943, -0.33422542, -0.07733303,
       -0.05552815,  0.13349698,  0.15030839, -0.15660381, -0.24477176,
        0.07735987,  0.04587603, -0.14181733,  0.08060279,  0.04808345,
       -0.10767551,  0.13980703, -0.10440759, -0.11982111,  0.12235789,
       -0.16645856,  0.01852177, -0.00102426, -0.03302133,  0.05966326,
       -0.36508772, -0.14287128,  0.21138044, -0.26033118,  0.12

array([-0.32080868, -0.44208628, -1.9293183 ,  0.52698386,  1.0503274 ,
       -0.4278534 , -0.46762788, -1.185181  ,  0.94758856,  0.8573701 ,
       -0.10250252, -0.44488475,  0.17767504,  0.33515498,  0.5420199 ,
       -1.144247  ,  0.5452122 , -0.2690067 , -1.4898711 , -0.49906024,
        1.3199672 ,  0.6627699 , -0.6692321 ,  0.12848899,  0.8057578 ,
       -1.213073  , -2.3001816 ,  0.6786487 , -1.6315558 , -0.69076014,
        1.2474767 ,  2.688198  , -0.4066019 , -0.6906432 , -1.6458328 ,
       -0.4838479 , -0.22592361,  0.50402784,  2.0284889 ,  0.02295262,
       -0.18683141, -0.06759565,  1.4908113 , -0.7008959 , -0.16842826,
       -0.9167728 ,  1.3537824 , -0.72132736, -1.966232  ,  0.58001393,
        0.235718  ,  1.2591712 ,  1.676096  , -0.00720707, -0.68636155,
       -0.09933744, -0.63568777, -1.018072  ,  0.47194186, -0.61218387,
        0.99325264,  0.13874184, -0.062783  ,  1.1817472 , -0.15142146,
       -0.6144181 , -1.7649287 ,  0.53576696, -1.0139626 ,  0.97

In [12]:
# Check word similarity

print("Top 5 words similar to word 'sound'")
model.wv.most_similar('sound', topn=5)

print("Top 5 words similar to word 'sound' but not relevant to 'film'")
model.wv.most_similar(positive=['sound','music'], negative=['film'], topn=5)

print("Similarity between 'movie' and 'film':")
model.wv.similarity('movie','film') 

print("Similarity between 'movie' and 'city':")
model.wv.similarity('movie','city') 

2018-11-29 16:50:45,587 : INFO : precomputing L2-norms of word weight vectors


Top 5 words similar to word 'sound'


[('sounding', 0.4111652970314026),
 ('noise', 0.39860135316848755),
 ('sounds', 0.3959551453590393),
 ('camera', 0.3572186231613159),
 ('voice', 0.35401657223701477)]

Top 5 words similar to word 'sound' but not relevant to 'film'


[('sounds', 0.3940489888191223),
 ('voice', 0.3609972298145294),
 ('noise', 0.34899282455444336),
 ('ballads', 0.34382694959640503),
 ('melodies', 0.3400244116783142)]

Similarity between 'movie' and 'film':


0.66123486

Similarity between 'movie' and 'city':


0.045854423

In [ ]:
# Inspect document similarity

model.docvecs.most_similar('0')

## 5. Convolutional Neural Networks (CNN)
References (**highly recommended**): 
- http://www.wildml.com/2015/11/understanding-convolutional-neural-networks-for-nlp/
- https://adeshpande3.github.io/adeshpande3.github.io/A-Beginner%27s-Guide-To-Understanding-Convolutional-Neural-Networks/

- CNNs are widely used in Computer Vision
- CNNs were responsible for **major breakthroughs** in **Image Recognition** and are the core of most Computer Vision systems including automated photo tagging, self-driving cars
- Recently, CNNs have been applied in NLP and achieved good performance.
<img src='cnn.png' width='90%'>

### 5.1. Convolution
- Convolution is the technique to **extract distinguishing features** from feature spaces
- Example: feature detection from image pixels
  - Feature space: a matrix of pixels of 0 (black) or 1 (white)
  - **Filter/kernal/feature Detector**: a function applied to every fixed subset of the feature matrix
    - e.g. 3x3 filter (a 3x3 matrix $\begin{vmatrix}
1 & 0 & 1 \\
0 & 1 & 0 \\
1 & 0 & 1
\end{vmatrix}$ ) slides through every area of the matrix sequentially, multiplies its values element-wise with the original matrix, then sum them up
    - e.g. a filter (e.g. $\begin{vmatrix}
0 & -1 & 0 \\
-1 & 4 & -1 \\
0 & -1 & 0
\end{vmatrix}$ ) to take difference between a pixel and its neighbors --> detect edges
    <img src='convolution.png' width="60%">
- Typically, a larger number of filters in different sizes will be used
- Configuration of filters
  - filter size ($h \text{x} w$)
  - stride size (how much to shift a filter in each step) ($s$)
  - number of filters (depth) ($d$)
- Questions: 
  - With 5x5 feature space, afte apply a filter of size 3x3 with stride size 2, what will be the size of the result? 
  - Formula to calculate the size?

### 5.2. Pooling Layer
- Pooling layers are typically applied after the convolutional layers. 
- A pooling layer subsamples its input. 
- The most common way to do pooling is to apply a **max** operation to the result of each filter (a.k.a 1-max pooling).
  - e.g. for the example below, by 1-max pooling, we get 8.
  - If 100 filters have been used, then we get 100 numbers
- Pooling can be applied over a window (e.g. 2x2)
<img src='max_pooling.png' width="60%">

### 5.3. What are CNNs
- CNNs consists of several layers of convolutions with nonlinear activation functions like ReLU or tanh 

<img src='cnn.png' width='70%'>

- A CNN typically contains:
  - A **convolution layer** (not dense layer) connected to the input layer
      - Each convolution layer applies different filters. 
      - Typically hundreds or thousands filters used. 
      - The results of filters are concatenated.
  - A **pooling layer** is used to subsample the result of convolution layer
  - There may be multiple layers of convolution and pooling combined. E.g. image detection
    - 1st layer: detect edges
    - 2nd layer: detect shape, e.g. round, square
    - 3rd layer: wheels, doors etc.
  - Then each result out of convolution-pooling is connected to a neuron in the output (local connections). Such  results results are high-level features used by classification algorithms. 
- During the training phase, a CNN **automatically learns the values of its filters based on the task you want to perform**. 
- Powerful capabilities of CNN:
  - **Location Invariance**: CNN extracts distinguishing features by convolution-pooling and it does not care where these features are. So images can still be recognized after rotation and scaling.
  - **Compositionality**: Each filter composes a local patch of lower-level features into higher-level representation. E.g., detect edges from pixels, shapes from edges, and more complex objects from shapes. 
- If you're interested in how CNNs are used in image recognition, follow the classical <a href="https://machinelearningmastery.com/handwritten-digit-recognition-using-convolutional-neural-networks-python-keras/">MNIST handwritten digit recognition tutorial</a>
- Play with it! http://scs.ryerson.ca/~aharley/vis/conv/flat.html

### 5.4. Application of CNN in Text Classification
- Assume $m$ samples, each of which is a sentence with $n$ words (short sentences can be padded)
- **Embedding**: In each sentence, each word can be represented as its word vector of dimension $d$ (pretrained or to be trained)
- **Convolution**: Apply filters to n-grams of different lengths (e.g. unigram, bigrams, ...). 
   - E.g. A filter can slide through every 2 words (bigram)
   - So, the filter size (i.e. region size) can be $1\text{x}d$ (unigram), $2\text{x}d$ (bigram), $3\text{x}d$ (trigram), ...
- At pooling layer, 1-max pooling is applied to the result of each filter. Then all results after pooling are concatenated as the input to the output layer
  - This is equivalent to select words or phrases that are **discriminative** with regard to the classification goal

<img src='cnn_text_classification.png' width='70%'>

*Illustration of a Convolutional Neural Network (CNN) architecture for sentence classification. Here we depict three filter region sizes: 2, 3 and 4, each of which has 2 filters. Every filter performs convolution on the sentence matrix and generates (variable-length) feature maps. Then 1-max pooling is performed over each map, i.e., the largest number from each feature map is recorded. Thus a univariate feature vector is generated from all six maps, and these 6 features are concatenated to form a feature vector for the penultimate layer. The final softmax layer then receives this feature vector as input and uses it to classify the sentence; here we assume binary classification and hence depict two possible output states. Source: Zhang, Y., & Wallace, B. (2015). A Sensitivity Analysis of (and Practitioners’ Guide to) Convolutional Neural Networks for Sentence Classification.*

- Questions:
  - How many parameters in total in the convolution layer?

### 5.5. How to deal with overfitting - Regularization & Dropout
- Deep neural nets with a large number of parameters can be easily suffer from overfitting 
- Typical approaches to overcome overfitting
    - Regularization
    - Dropout (which is also a kind of regularization technique)
- What is dropout?
  - During training, randomly remove units in the hidden layer from the network. Update parameters as normal, leaving dropped-out units unchanged
  - No dropout during testing 
  - Typically, each hidden unit is set to 0 with probability 0.5
  <img src='dropout.png' width='60%'>
  https://www.cs.toronto.edu/~hinton/absps/JMLRdropout.pdf
 
- Why dropout?
  - Hidden units cannot co-adapt with other units since a unit may not always be present 
  - Sample data usually come with noise. Dropout constrains network adaptation to the data at training time
  - After training, only very useful neurons are kept (have high weights) 

### 5.6. Example: Use CNN for Sentiment Analysis (Single-Label Classification)
- Dataset: IMDB review
- 25,000 movie reviews, positive or negative 
- Benchmark performance is 80-90% with CNN (https://arxiv.org/abs/1408.5882)
- We're going to create a CNN with the following:
  - Word embedding trained as part of CNN
  - filters in 3 sizes:
      - unigram (Conv1D, kernel_size=1)
      - bigram (Conv1D, kernel_size=2)
      - trigram (Conv1D, kernel_size=3)
  - Maxpooling for each convolution layer
  - Dropout
  <img src="cnn_model.png" width='60%'>

In [ ]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [ ]:
# Exercise 5.1: Load data

import pandas as pd
import nltk,string
from gensim import corpora

data=pd.read_csv("../../../dataset/imdb_reviews.csv", header=0, delimiter="\t")
data.head()
len(data)

# if your computer does not have enough resource
# reduce the dataset
data=data.loc[0:8000]

In [ ]:
# Exercise 5.2 Prepocessing data: Tokenize, pad sentences

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
 
import numpy as np

# set the maximum number of words to be used
MAX_NB_WORDS=10000

# set sentence/document length
MAX_DOC_LEN=500

# get a Keras tokenizer
# https://keras.io/preprocessing/text/
tokenizer = Tokenizer(num_words=MAX_NB_WORDS)
tokenizer.fit_on_texts(data["review"])

# convert each document to a list of word index as a sequence
sequences = tokenizer.texts_to_sequences(data["review"])

# pad all sequences into the same length 
# if a sentence is longer than maxlen, pad it in the right
# if a sentence is shorter than maxlen, truncate it in the right
padded_sequences = pad_sequences(sequences, \
                                 maxlen=MAX_DOC_LEN, \
                                 padding='post', \
                                 truncating='post')

print(padded_sequences[0])

In [ ]:
# get the mapping between word and its index
tokenizer.word_index['film']

# get the count of each word
tokenizer.word_counts['film']

In [ ]:
# Split data for training and testing

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(\
                        padded_sequences, data['sentiment'],\
                        test_size=0.3, random_state=1)

In [ ]:
# Exercise 5.3: Create CNN model

from keras.layers import Embedding, Dense, Conv1D, MaxPooling1D, \
Dropout, Activation, Input, Flatten, Concatenate
from keras.models import Model

# The dimension for embedding
EMBEDDING_DIM=100

# define input layer, where a sentence represented as
# 1 dimension array with integers
main_input = Input(shape=(MAX_DOC_LEN,), \
                   dtype='int32', name='main_input')

# define the embedding layer
# input_dim is the size of all words +1
# where 1 is for the padding symbol
# output_dim is the word vector dimension
# input_length is the max. length of a document
# input to embedding layer is the "main_input" layer
embed_1 = Embedding(input_dim=MAX_NB_WORDS+1, \
                    output_dim=EMBEDDING_DIM, \
                    input_length=MAX_DOC_LEN,\
                    name='embedding')(main_input)


# define 1D convolution layer
# 64 filters are used
# a filter slides through each word (kernel_size=1)
# input to this layer is the embedding layer
conv1d_1= Conv1D(filters=64, kernel_size=1, \
                 name='conv_unigram',\
                 activation='relu')(embed_1)

# define a 1-dimension MaxPooling 
# to take the output of the previous convolution layer
# the convolution layer produce 
# MAX_DOC_LEN-1+1 values as ouput (???)
pool_1 = MaxPooling1D(MAX_DOC_LEN-1+1, \
                      name='pool_unigram')(conv1d_1)

# The pooling layer creates output 
# in the size of (# of sample, 1, 64)  
# remove one dimension since the size is 1
flat_1 = Flatten(name='flat_unigram')(pool_1)

# following the same logic to define 
# filters for bigram
conv1d_2= Conv1D(filters=64, kernel_size=2, \
                 name='conv_bigram',\
                 activation='relu')(embed_1)
pool_2 = MaxPooling1D(MAX_DOC_LEN-2+1, name='pool_bigram')(conv1d_2)
flat_2 = Flatten(name='flat_bigram')(pool_2)

# filters for trigram
conv1d_3= Conv1D(filters=64, kernel_size=3, \
                 name='conv_trigram',activation='relu')(embed_1)
pool_3 = MaxPooling1D(MAX_DOC_LEN-3+1, name='pool_trigram')(conv1d_3)
flat_3 = Flatten(name='flat_trigram')(pool_3)

# Concatenate flattened output
z=Concatenate(name='concate')([flat_1, flat_2, flat_3])

# Create a dropout layer
# In each iteration only 50% units are turned on
drop_1=Dropout(rate=0.5, name='dropout')(z)

# Create a dense layer
dense_1 = Dense(192, activation='relu', name='dense')(drop_1)
# Create the output layer
preds = Dense(1, activation='sigmoid', name='output')(dense_1)

# create the model with input layer
# and the output layer
model = Model(inputs=main_input, outputs=preds)

In [ ]:
# Exercise 5.4: Show model configuration

model.summary()
#model.get_config()
#model.get_weights()
#from keras.utils import plot_model
#plot_model(model, to_file='cnn_model.png')

In [ ]:
# Exercise 5.4: Compile the model

model.compile(loss="binary_crossentropy", \
              optimizer="adam", \
              metrics=["accuracy"])

In [ ]:
# Exercise 5.5: Fit the model

BATCH_SIZE = 64
NUM_EPOCHES = 10

# fit the model and save fitting history to "training"
training=model.fit(X_train, y_train, \
                   batch_size=BATCH_SIZE, \
                   epochs=NUM_EPOCHES,\
                   validation_data=[X_test, y_test], \
                   verbose=2)

In [ ]:
# Exercise 5.6. Investigate the training process

import matplotlib.pyplot as plt
import pandas as pd
# plot a figure with size 20x8

# the fitting history is saved as dictionary
# covert the dictionary to dataframe
df=pd.DataFrame.from_dict(training.history)
df.columns=["train_acc", "train_loss", \
            "val_acc", "val_loss"]
df.index.name='epoch'
print(df)

# plot training history
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(8,3));

df[["train_acc", "val_acc"]].plot(ax=axes[0]);
df[["train_loss", "val_loss"]].plot(ax=axes[1]);
plt.show();

Observations from training history:
- As training goes on, training accuracy/loss gets always better
- Testing accuracy/loss gets better at the beginning, the gets worse
- This indicates that model is **overfitted** and cannot be generalized after certain point
- Thus, we should **stop training the model when testing accuracy/loss gets worse**. 
- This analysis can be used to determine hyperparameter **NUM_EPOCHES**
- Fortunately, this can be done automatically by **"Early Stopping"**

In [ ]:
# Exercise 5.6: Use early stopping to find the best model

from keras.callbacks import EarlyStopping, ModelCheckpoint

# the file path to save best model
BEST_MODEL_FILEPATH="best_model"

# define early stopping based on validation loss
# if validation loss is not improved in 
# an iteration compared with the previous one, 
# stop training (i.e. patience=0). 
# mode='min' indicate the loss needs to decrease 
earlyStopping=EarlyStopping(monitor='val_loss', \
                            patience=0, verbose=2, \
                            mode='min')

# define checkpoint to save best model
# which has max. validation acc
checkpoint = ModelCheckpoint(BEST_MODEL_FILEPATH, \
                             monitor='val_acc', \
                             verbose=2, \
                             save_best_only=True, \
                             mode='max')

# compile model
model.compile(loss="binary_crossentropy", \
              optimizer="adam", metrics=["accuracy"])

# fit the model with earlystopping and checkpoint
# as callbacks (functions that are executed as soon as 
# an asynchronous thread is completed)
model.fit(X_train, y_train, \
          batch_size=BATCH_SIZE, epochs=NUM_EPOCHES, \
          callbacks=[earlyStopping, checkpoint],
          validation_data=[X_test, y_test],\
          verbose=2)

In [ ]:
# Exercise 5.7: Load the best model

# load the model using the save file
model.load_weights("best_model")

# predict
pred=model.predict(X_test)
print(pred[0:5])
# evaluate the model
scores = model.evaluate(X_test, y_test, verbose=0)
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

In [ ]:
# Exercise 5.8: Put Everything as a function

from keras.layers import Embedding, Dense, Conv1D, MaxPooling1D, \
Dropout, Activation, Input, Flatten, Concatenate
from keras.models import Model
from keras.regularizers import l2
from keras.callbacks import EarlyStopping, ModelCheckpoint
              
def cnn_model(FILTER_SIZES, \
              # filter sizes as a list
              MAX_NB_WORDS, \
              # total number of words
              MAX_DOC_LEN, \
              # max words in a doc
              EMBEDDING_DIM=200, \
              # word vector dimension
              NUM_FILTERS=64, \
              # number of filters for all size
              DROP_OUT=0.5, \
              # dropout rate
              NUM_OUTPUT_UNITS=1, \
              # number of output units
              NUM_DENSE_UNITS=100,\
              # number of units in dense layer
              PRETRAINED_WORD_VECTOR=None,\
              # Whether to use pretrained word vectors
              LAM=0.0):            
              # regularization coefficient
    
    main_input = Input(shape=(MAX_DOC_LEN,), \
                       dtype='int32', name='main_input')
    
    if PRETRAINED_WORD_VECTOR is not None:
        embed_1 = Embedding(input_dim=MAX_NB_WORDS+1, \
                        output_dim=EMBEDDING_DIM, \
                        input_length=MAX_DOC_LEN, \
                        # use pretrained word vectors
                        weights=[PRETRAINED_WORD_VECTOR],\
                        # word vectors can be further tuned
                        # set it to False if use static word vectors
                        trainable=True,\
                        name='embedding')(main_input)
    else:
        embed_1 = Embedding(input_dim=MAX_NB_WORDS+1, \
                        output_dim=EMBEDDING_DIM, \
                        input_length=MAX_DOC_LEN, \
                        name='embedding')(main_input)
    # add convolution-pooling-flat block
    conv_blocks = []
    for f in FILTER_SIZES:
        conv = Conv1D(filters=NUM_FILTERS, kernel_size=f, \
                      activation='relu', name='conv_'+str(f))(embed_1)
        conv = MaxPooling1D(MAX_DOC_LEN-f+1, name='max_'+str(f))(conv)
        conv = Flatten(name='flat_'+str(f))(conv)
        conv_blocks.append(conv)
    
    if len(conv_blocks)>1:
        z=Concatenate(name='concate')(conv_blocks)
    else:
        z=conv_blocks[0]
        
    drop=Dropout(rate=DROP_OUT, name='dropout')(z)

    dense = Dense(NUM_DENSE_UNITS, activation='relu',\
                    kernel_regularizer=l2(LAM),name='dense')(drop)
    preds = Dense(NUM_OUTPUT_UNITS, activation='sigmoid', name='output')(dense)
    model = Model(inputs=main_input, outputs=preds)
    
    model.compile(loss="binary_crossentropy", \
              optimizer="adam", metrics=["accuracy"]) 
    
    return model

### 5.7. Use CNN for multi-label classification
- In multi-label classification, a document can be classified into multiple classes
- We can use **multiple ouput units**, each responsible for predicating one class
- For multi-label classification ($K$ classes), do the following:
    1. Represent the labels as **indication matrix**
        - e.g. three classes ['econ','biz','tech'] in total, 
        - sample 1: 'eco' only -> [1, 0, 0]
        - sample 2: ['eco','biz'] ->[1, 1, 0]
    2. Accordingly, **set output layer to have K output units**
        - each responsible for one class
        - each unit gives the probabability of one class


- Example: Yahoo News Ranked Multilabel Learning dataset (http://research.yahoo.com)
  - A subset is selected
  - 4 classes, 6426 samples

In [ ]:
# Exercise 5.7.1: Load and process the data

import json
from sklearn.preprocessing import MultiLabelBinarizer
from numpy.random import shuffle

# load the data
data=json.load(open("../../dataset/ydata.json",'rb'))
#data=json.load(open("ydata.json",'r'))


# shuffle the data
shuffle(data)

# split into text and label
text,labels=zip(*data)
text=list(text)
labels=list(labels)
text[1]
labels[1]

In [ ]:
# Exercise 5.7.2: create indicator matrix for labels

mlb = MultiLabelBinarizer()
Y=mlb.fit_transform(labels)
# check size of indicator matrix
Y.shape
# check classes
mlb.classes_
# check # of samples in each class
np.sum(Y, axis=0)

In [ ]:
# Exercise 5.7.3: Load and process the data

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
 
import numpy as np

# get a Keras tokenizer

MAX_NB_WORDS=8000
# documents are quite long in the dataset
MAX_DOC_LEN=1000

tokenizer = Tokenizer(num_words=MAX_NB_WORDS)
tokenizer.fit_on_texts(text)
voc=tokenizer.word_index
# convert each document to a list of word index as a sequence
sequences = tokenizer.texts_to_sequences(text)
# get the mapping between words to word index

# pad all sequences into the same length (the longest)
padded_sequences = pad_sequences(sequences, \
                                 maxlen=MAX_DOC_LEN, \
                                 padding='post', truncating='post')

#print(padded_sequences[0])


In [ ]:
# Exercise 5.7.4: Fit the model using the function

from sklearn.model_selection import train_test_split

EMBEDDING_DIM=100
FILTER_SIZES=[2,3,4]

# set the number of output units
# as the number of classes
output_units_num=len(mlb.classes_)
num_filters=64

# set the dense units
dense_units_num= num_filters*len(FILTER_SIZES)


BTACH_SIZE = 64
NUM_EPOCHES = 20

# split dataset into train (70%) and test sets (30%)
X_train, X_test, Y_train, Y_test = train_test_split(\
                padded_sequences, Y, test_size=0.2, random_state=0)


model=cnn_model(FILTER_SIZES, MAX_NB_WORDS, \
                MAX_DOC_LEN, \
                NUM_FILTERS=num_filters,\
                NUM_OUTPUT_UNITS=output_units_num, \
                NUM_DENSE_UNITS=dense_units_num)

earlyStopping=EarlyStopping(monitor='val_loss', patience=0, verbose=2, mode='min')
checkpoint = ModelCheckpoint(BEST_MODEL_FILEPATH, monitor='val_loss', \
                             verbose=2, save_best_only=True, mode='min')
    
training=model.fit(X_train, Y_train, \
          batch_size=BTACH_SIZE, epochs=NUM_EPOCHES, \
          callbacks=[earlyStopping, checkpoint],\
          validation_data=[X_test, Y_test], verbose=2)

In [ ]:
# Exercise 5.7.5: predicate using the best model
# calculate performance

# load the best model
model.load_weights("best_model")

pred=model.predict(X_test)
pred[0:5]

In [ ]:
# Exercise 5.7.6: Generate performance report
from sklearn.metrics import classification_report

pred=np.where(pred>0.5, 1, 0)

print(classification_report(Y_test, pred,\
                      target_names=mlb.classes_))


### 5.8.  Use Pretrained Word Vectors
- If **the size of labeled samples is small, it's better use pretrained word vectors** 
    - e.g. google or facebook pretrained word vectors
    - or you can train word vectors using relevant context data using gensim
- Procedure:
    1. Obtain/train pretrained word vectors (see Section 4.1 and Exercise 4.1.1)
    2. Look for the word vector for each word in the vocabulary and create **embedding matrix** where each row represents one word vector
    3. Set embedding layer with the embedding matrix and set it not trainable.
- With well-trained word vectors, often a small sample set can also achieve good performance

In [ ]:
# Exercise 5.8.1: Load full yahoo news dataset
# to train the word vector
# note this data can be unlabeled. only text is used
import json

data=json.load(open("../../dataset/ydata_full.json",'r'))
text,labels=zip(*data)
text=list(text)

sentences=[ [token.strip(string.punctuation).strip() \
             for token in nltk.word_tokenize(doc) \
                 if token not in string.punctuation and \
                 len(token.strip(string.punctuation).strip())>=2]\
             for doc in text]

In [ ]:
# Exercise 5.8.2: Train word vector using 
# the large data set

from gensim.models import word2vec
import logging
import pandas as pd

# print out tracking information
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', \
                    level=logging.INFO)
EMBEDDING_DIM=200
# min_count: words with total frequency lower than this are ignored
# size: the dimension of word vector
# window: is the maximum distance 
#         between the current and predicted word 
#         within a sentence (i.e. the length of ngrams)
# workers: # of parallel threads in training
# for other parameters, check https://radimrehurek.com/gensim/models/word2vec.html
wv_model = word2vec.Word2Vec(sentences, \
                             min_count=5, \
                             size=EMBEDDING_DIM, \
                             window=5, workers=4 )

In [ ]:
# get word vector for all words in the vocabulary
# see reference at https://github.com/fchollet/keras/blob/master/examples/pretrained_word_embeddings.py

EMBEDDING_DIM=200
MAX_NB_WORDS=8000

# tokenizer.word_index provides the mapping 
# between a word and word index for all words
NUM_WORDS = min(MAX_NB_WORDS, len(tokenizer.word_index))

# "+1" is for padding symbol
embedding_matrix = np.zeros((NUM_WORDS+1, EMBEDDING_DIM))

for word, i in tokenizer.word_index.items():
    # if word_index is above the max number of words, ignore it
    if i >= NUM_WORDS:
        continue
    if word in wv_model.wv:
        embedding_matrix[i]=wv_model.wv[word]

In [ ]:
# Exercise 5.8.3: Fit model using pretrained word vectors

from sklearn.model_selection import train_test_split

EMBEDDING_DIM=200
FILTER_SIZES=[2,3,4]

# set the number of output units
# as the number of classes
output_units_num=len(mlb.classes_)

#Number of filters for each size
num_filters=64

# set the dense units
dense_units_num= num_filters*len(FILTER_SIZES)

BTACH_SIZE = 32
NUM_EPOCHES = 100

# With well trained word vectors, sample size can be reduced
# Assume we only have 500 labeled data
# split dataset into train (80%) and test sets (20%)

X_train, X_test, Y_train, Y_test = train_test_split(\
                padded_sequences[0:500], Y[0:500], \
                test_size=0.2, random_state=0, \
                shuffle=True)

# create the model with embedding matrix
model=cnn_model(FILTER_SIZES, MAX_NB_WORDS, \
                MAX_DOC_LEN, \
                NUM_FILTERS=num_filters,\
                NUM_OUTPUT_UNITS=output_units_num, \
                NUM_DENSE_UNITS=dense_units_num,\
                PRETRAINED_WORD_VECTOR=embedding_matrix)

earlyStopping=EarlyStopping(monitor='val_loss', patience=1, verbose=2, mode='min')
checkpoint = ModelCheckpoint(BEST_MODEL_FILEPATH, monitor='val_loss', \
                             verbose=2, save_best_only=True, mode='min')
    
training=model.fit(X_train, Y_train, \
          batch_size=BTACH_SIZE, epochs=NUM_EPOCHES, \
          callbacks=[earlyStopping, checkpoint],\
          validation_data=[X_test, Y_test], verbose=2)

In [ ]:
# Exercise 5.8.4: check model configuration
# Note that parameters from embedding layer
# is not trainable

model.summary()

In [ ]:
# Exercise 5.8.5: Performance evaluation
# Let's use samples[500:1000]
# as an evaluation set

from sklearn.metrics import classification_report

pred=model.predict(padded_sequences[500:1000])

Y_pred=np.copy(pred)
Y_pred=np.where(Y_pred>0.5,1,0)

Y_pred[0:10]
Y[500:510]

print(classification_report(Y[500:1000], \
                Y_pred, target_names=mlb.classes_))

Observations:
- Note that we only trained the model with **500 samples**
- The performance is only slightly lower, compared with the one trained with 6000 samples
- This shows that pre-trained word vectors can effectively improve the classification performance in the case of small labeled dataset

### 5.9. How to select hyperparameters?
- Fitting a neural network is a very empirical process
- See Section 3 of "Practical Recommendations for Gradient-Based Training of Deep Architectures" (https://arxiv.org/abs/1206.5533) for detailed discussion
- The following is some useful techniques to set 
  - MAX_NB_WORDS: max number words to be included in word embedding
    - Based on word frequency histogram to include words that appear at least $n$ times
  - MAX_DOC_LEN: max length of documents
    - Based on document length frequency histogram to include complete sentences as many as possible


In [ ]:
# Exercise 5.9.1 Set MAX_NB_WORDS to 
# include words that appear at least K times

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

# get count of each word
df=pd.DataFrame.from_dict(tokenizer.word_counts, \
                          orient="index")
df.columns=['freq']
print(df.head())

# get histogram of word count
df=df['freq'].value_counts().reset_index()
df.columns=['word_freq','count']

# sort by word_freq
df=df.sort_values(by='word_freq')

# convert absolute counts to precentage
df['percent']=df['count']/len(tokenizer.word_counts)
# get cumulative percentage
df['cumsum']=df['percent'].cumsum()

print(df.head())

df.iloc[0:50].plot(x='word_freq', y='cumsum');

plt.show();

# if set min count for word to 10, 
# what % of words can be included?
# how many words will be included?
# This is the parameter MAX_NB_WORDS
# tokenizer = Tokenizer(num_words=MAX_NB_WORDS)

In [ ]:
# Exercise 5.9.2 Set MAX_DOC_LEN to 
# include complete sentences as many as possible

# create a series based on the length of all sentences
sen_len=pd.Series([len(item) for item in sequences])

# create histogram of sentence length
# the "index" is the sentence length
# "counts" is the count of sentences at a length
df=sen_len.value_counts().reset_index().sort_values(by='index')
df.columns=['sent_length','counts']

# sort by sentence length
# get percentage and cumulative percentage

df['percent']=df['counts']/len(sen_len)
df['cumsum']=df['percent'].cumsum()
print(df.head(3))

# From the plot, 90% sentences have length<500
# so it makes sense to set MAX_DOC_LEN=4~500 
df.plot(x="sent_length", y='cumsum');
plt.show();

# what will be the minimum sentence length
# such that 99% of sentences will not be truncated?